In [1]:
# imports
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
import cv2
import tensorflow as tf
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow import data as tf_data
import time
from sklearn.metrics import confusion_matrix, precision_score, recall_score, log_loss
import numpy as np
from typing import List, Dict, Any, Union

from utils import print_progress_bar
print("Modules imported")

Modules imported


In [2]:
# variables
all_labels = ['nature', 'country', 'city']
path = "../datasets/all_data/entropy_results_short.json"
parallel_jobs = 5

# hyperparameters
test_part = 0.05
epochs = 2
batch_size = 64
learning_rate = 0.01

# check gpu
devices = tf.config.list_physical_devices()
print("Available devices:")
for device in devices:
    print(device.name)
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print(f"The model will run on GPU: {physical_devices[0].name}")
else:
    print("No GPU found, the model will run on CPU.")

Available devices:
/physical_device:CPU:0
/physical_device:GPU:0
The model will run on GPU: /physical_device:GPU:0


In [9]:
# heads
class SelfAttention(tf.keras.layers.Layer):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"

        self.wq = tf.keras.layers.Dense(self.head_dim)
        self.wk = tf.keras.layers.Dense(self.head_dim)
        self.wv = tf.keras.layers.Dense(self.head_dim)

    def call(self, inputs):
        Q = self.wq(inputs)
        K = self.wk(inputs)
        V = self.wv(inputs)

        matmul_qk = tf.matmul(Q, K, transpose_b=True)

        depth = tf.cast(tf.shape(K)[-1], tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)

        attention_weights = tf.nn.softmax(logits, axis=-1)

        output = tf.matmul(attention_weights, V)
        return output

In [ ]:
# model
class EntropyClassifier(tf.keras.Model):
    def __init__(self, possible_labels):
        super(EntropyClassifier, self).__init__()

        self.possible_labels = possible_labels

        self.dwt_input_layer = tf.keras.layers.Dense(10, activation='relu')
        self.lvl0_input_layer = tf.keras.layers.Dense(17, activation='relu')
        self.lvl1_input_layers = [tf.keras.layers.Conv2D(1, (2, 2), activation='relu') for _ in range(17)]
        self.lvl2_input_layers = [tf.keras.layers.Conv2D(1, (2, 2), activation='relu') for _ in range(17)]
        self.lvl3_input_layers = [tf.keras.layers.Conv2D(1, (2, 2), activation='relu') for _ in range(17)]

    def call(self, inputs):
        lvl0_inputs, lvl1_inputs, lvl2_inputs, lvl3_inputs, dwt_inputs = inputs
        dwt_output = self.dwt_input_layer(dwt_inputs)
        lvl0_output = self.lvl0_input_layer(lvl0_inputs)
    
        lvl1_output = tf.concat([self.lvl1_input_layers[i](tf.expand_dims(tf.expand_dims(lvl1_inputs[:,:,i], axis=0), axis=-1)) for i in range(17)], axis=-1)
        lvl2_output = tf.concat([self.lvl2_input_layers[i](tf.expand_dims(tf.expand_dims(lvl2_inputs[:,:,i], axis=0), axis=-1)) for i in range(17)], axis=-1)
        lvl3_output = tf.concat([self.lvl3_input_layers[i](tf.expand_dims(tf.expand_dims(lvl3_inputs[:,:,i], axis=0), axis=-1)) for i in range(17)], axis=-1)
        
        print(dwt_output.shape)
        print(lvl0_output.shape)
        print(lvl1_output.shape)
        print(lvl2_output.shape)
        print(lvl3_output.shape)

In [4]:
def process_entry(entry):
    """Process the entropy results to extract the levels."""
    label = entry['label']
    machine_input = {0: [], 1: [], 2: [], 3: [], 'dwt': []}
    for ent in entry['entropy_results']:
    
        if ent['method'] == 'dwt':
            machine_input['dwt'] = tf.convert_to_tensor(ent['result'], dtype=tf.float32)
        else:
            for lvl, content in enumerate(ent['result']):
                machine_input[lvl].append(tf.convert_to_tensor(content, dtype=tf.float32))
    
    machine_input[0] = tf.concat(machine_input[0], axis=-1)
    machine_input[1] = tf.concat(machine_input[1], axis=-1)
    machine_input[2] = tf.concat(machine_input[2], axis=-1)
    machine_input[3] = tf.concat(machine_input[3], axis=-1)
    machine_input['dwt'] = tf.reshape(machine_input['dwt'], [1, 1, 10])
    
    return {'input': machine_input, 'label': label}  


def format_dataset(dataset):
    """Formats and shuffles the dataset for training"""
    label_num = {'nature': 0, 'country': 1, 'city': 2}
    formatted_dataset = []
    for item in dataset:
        machine_input = item['input']
        label = label_num[item['label']]
        formatted_dataset.append((
            (
                machine_input[0], 
                machine_input[1], 
                machine_input[2], 
                machine_input[3], 
                machine_input['dwt']
            ), 
            label
        ))
    return tf.data.Dataset.from_generator(
        lambda: iter(formatted_dataset), 
        output_signature=(
            (
                tf.TensorSpec(shape=(1, 1, 17), dtype=tf.float32),
                tf.TensorSpec(shape=(2, 2, 17), dtype=tf.float32),
                tf.TensorSpec(shape=(4, 4, 17), dtype=tf.float32),
                tf.TensorSpec(shape=(8, 8, 17), dtype=tf.float32),
                tf.TensorSpec(shape=(1, 1, 10), dtype=tf.float32)
            ),
            tf.TensorSpec(shape=(), dtype=tf.int32),
        )
    ).shuffle(buffer_size=len(dataset))


def process_json(path, test_part, parallel_jobs=4):
    """Process JSON data to extract dataset and features."""
    with open(path, 'r') as f:
        metadata = json.load(f)
    dataset = []

    t = time.time()
    n = len(metadata)

    with ThreadPoolExecutor(max_workers=parallel_jobs) as executor:
        futures = [executor.submit(process_entry, entry) for entry in metadata]
        for i, future in enumerate(as_completed(futures)):
            result = future.result()
            if result is not None:
                dataset.append(result)
            print_progress_bar('Processed entry', i+1, n, t)

    if isinstance(test_part, float):
        i = int(test_part * len(dataset))
    elif isinstance(test_part, str):
        i = int(test_part)
    else:
        raise ValueError("Incompatible format for 'test_part'.")

    test_set = dataset[-i:]
    dataset = dataset[:-i]

    num_classes = len(all_labels)
    dataset_length = len(dataset)

    return dataset, test_set, num_classes, dataset_length


In [5]:
# load data
dataset, test_set, num_classes, dataset_length = process_json(path, test_part)
print('\nDataset processed.')
print(f"Total number of entries in the dataset: {dataset_length}")
print(f"Total number of entries in the test set: {len(test_set)}")    
print(f"Number of classes: {num_classes}")
print(f"Dataset length: {dataset_length}") 

Processed entry: ██████████████████████████████████████████████████ | Completed: 256/256 100.0% | Time elapsed: 00:01/00:01 | Time left: ~= 00:00
Dataset processed.
Total number of entries in the dataset: 244
Total number of entries in the test set: 12
Number of classes: 3
Dataset length: 244


In [6]:
sample = dataset[0]
print(sample['label'])
for inp, val in sample['input'].items():
    print(f'level: {inp}. shape: {val.shape}')
machine_inputs = tuple(sample['input'].values())

nature
level: 0. shape: (1, 1, 17)
level: 1. shape: (2, 2, 17)
level: 2. shape: (4, 4, 17)
level: 3. shape: (8, 8, 17)
level: dwt. shape: (1, 1, 10)


In [7]:
# model creation
file_name = f"EntropyClassifier_e={epochs}_ds={dataset_length}.pth"
model = EntropyClassifier(all_labels)
print('Model created')

Model created


In [8]:
model.call(machine_inputs)

(1, 1, 10)
(1, 1, 17)
(1, 1, 1, 17)
(1, 3, 3, 17)
(1, 7, 7, 17)
